In [1]:
!pip install pyverilog
!apt-get install graphviz graphviz-dev
!pip install pygraphviz
!apt-get update
!apt-get install -y iverilog
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.4/136.4 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00
  Created wheel for pyverilog: filename=pyverilog-1.3.0-py3-none-any.whl size=118928 sha256=406351e24dc888316437fa84bc4ccab7fd380d5a3260ab84696f2e02c539a12d
  Stored in directory: /root/.cache/pip/wheels/cf/6e/86/42cdc93d68295ef50b38b8ad1fba394ac319f2b39c896b1975
Successfully built pyverilog
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'libgraphviz-dev' instead of 'graphviz-dev'
graphviz is already the newest version (2.42.2-6ubuntu0.1).
The following additional packages will be installed:
  libgail-common libgail18 libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libgvc6-plugins-gtk
  librsvg2-common libxdot4
Suggested packages:
  gvfs
The following NEW packages will be installed:
  libgail-common libgail18 libgraphviz-dev lib

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os, sys
sys.path.append('/content/drive/MyDrive/HW2VEC/')
from hw2vec.hw2graph import *
from hw2vec.config import Config
from hw2vec.graph2vec.models import *

Use Case 1:

In [4]:
args = [
    '--yaml_path', '/content/drive/MyDrive/HW2VEC/examples/example_gnn4tj.yaml',
    '--raw_dataset_path', '/content/drive/MyDrive/HW2VEC/assets/TJ-RTL-toy/',
    '--data_pkl_path', '/content/drive/MyDrive/HW2VEC/examples/dfg_tj_rtl.pkl',
    '--graph_type', 'DFG',
    '--device', 'cuda'
    ]
cfg = Config(args)

In [5]:
hw2graph = HW2GRAPH(cfg)

hw_design_path = hw2graph.preprocess(hw_design_dir_path)
hardware_nxgraph = hw2graph.process(hw_design_path)

data_proc = DataProcessor(cfg)
data_proc.process(hardware_nxgraph)
vis_loader = DataLoader(data_proc.get_graphs(), batch_size=1)

NameError: name 'hw_design_dir_path' is not defined

In [6]:
graph_data = next(iter(vis_loader)).to(cfg.device)
graph_embed, _ = model.embed_graph(graph_data.x, graph_data.edge_index, graph_data.batch)
print(graph_embed)

NameError: name 'vis_loader' is not defined

Use Case 2

In [7]:
args = [
    '--yaml_path', '/content/drive/MyDrive/HW2VEC/examples/example_gnn4tj.yaml',
    '--raw_dataset_path', '/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/',
    '--data_pkl_path', '/content/drive/MyDrive/HW2VEC/examples/dfg_tj_rtl.pkl',
    '--graph_type', 'DFG',
    '--device', 'cuda'
    ]
cfg = Config(args)

In [8]:
''' prepare graph data '''
print(cfg.raw_dataset_path)
for verilog_path in glob("%s/**/*.v" % str(cfg.raw_dataset_path), recursive=True):
    folder_name = Path(verilog_path).parent
    print(folder_name)

''' converting graph using hw2graph '''
nx_graphs = []
hw2graph = HW2GRAPH(cfg)
for hw_project_path in hw2graph.find_hw_project_folders():
    hw_graph = hw2graph.code2graph(hw_project_path)
    print(hw_graph)
    nx_graphs.append(hw_graph)

data_proc = DataProcessor(cfg)
for hw_graph in nx_graphs:
    data_proc.process(hw_graph)
data_proc.cache_graph_data(cfg.data_pkl_path)

/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/PIC16F84-T100
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T901
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T400
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T700
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/PIC16F84-T200
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T300
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T500
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/PIC16F84-T300
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T900
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T800
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/RS232-T200
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-toy/TjIn/PIC16F84-T400
/content/drive/MyDrive/HW2VEC/assets/datasets/TJ-RTL-to

In [9]:
''' prepare dataset '''
TROJAN = 1
NON_TROJAN = 0

all_graphs = data_proc.get_graphs()
print(all_graphs)
for data in all_graphs:
    if "TjFree" == data.hw_type:
        data.label = NON_TROJAN
    else:
        data.label = TROJAN

train_graphs, test_graphs = data_proc.split_dataset(ratio=cfg.ratio, seed=cfg.seed, dataset=all_graphs)
train_loader = DataLoader(train_graphs, shuffle=True, batch_size=cfg.batch_size)
valid_loader = DataLoader(test_graphs, shuffle=True, batch_size=1)

[Data(x=[32], edge_index=[2, 32], label=[32], name='syncRAM', hw_name='syncRAM', hw_type='TjFree'), Data(x=[2636], edge_index=[2, 3694], label=[2636], name='PIC16F84-T300', hw_name='PIC16F84-T300', hw_type='TjIn'), Data(x=[760], edge_index=[2, 891], label=[760], name='RS232-T100', hw_name='RS232-T100', hw_type='TjIn'), Data(x=[2541], edge_index=[2, 3576], label=[2541], name='PIC16F84', hw_name='PIC16F84', hw_type='TjFree'), Data(x=[833], edge_index=[2, 1034], label=[833], name='RS232-T600', hw_name='RS232-T600', hw_type='TjIn'), Data(x=[2636], edge_index=[2, 3694], label=[2636], name='PIC16F84-T400', hw_name='PIC16F84-T400', hw_type='TjIn'), Data(x=[717], edge_index=[2, 838], label=[717], name='RS232-T300', hw_name='RS232-T300', hw_type='TjIn'), Data(x=[144], edge_index=[2, 195], label=[144], name='spi_master', hw_name='spi_master', hw_type='TjFree'), Data(x=[2411], edge_index=[2, 2716], label=[2411], name='RC6', hw_name='RC6', hw_type='TjFree'), Data(x=[833], edge_index=[2, 1034], lab

In [10]:
''' model configuration '''
model = GRAPH2VEC(cfg)

convs = [
    GRAPH_CONV("gcn", data_proc.num_node_labels, cfg.hidden),
    GRAPH_CONV("gcn", cfg.hidden, cfg.hidden)
]
model.set_graph_conv(convs)

pool = GRAPH_POOL("sagpool", cfg.hidden, cfg.poolratio)
model.set_graph_pool(pool)

readout = GRAPH_READOUT("max")
model.set_graph_readout(readout)

output = nn.Linear(cfg.hidden, cfg.embed_dim)
model.set_output_layer(output)

In [11]:
''' training '''
training_labels = [data.label for data in train_graphs]
print(training_labels)
tl_class_weights = torch.tensor(compute_class_weight('balanced', classes=np.unique(training_labels), y=training_labels), dtype=torch.float)

model.to(cfg.device)
#trainer = GraphTrainer(cfg, class_weights=data_proc.get_class_weights(train_graphs))
trainer = GraphTrainer(cfg, class_weights=tl_class_weights)
trainer.build(model)
trainer.train(train_loader, valid_loader)

[1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1]


Epoch: 0003, loss_train: 3.2377:   2%|▏         | 4/200 [00:02<01:38,  1.99it/s]


Mini Test for Epochs 0:
train loss: 0.615332, train accuracy: 0.6111, train f1 score: 0.7586, train confusion_matrix: [[ 0  7], [ 0 11]], train precision: 0.6111, train recall: 1.0000
test  loss: 0.653310, test  accuracy: 0.6000, test  f1 score: 0.7500, test  confusion_matrix: [[0 2], [0 3]], test  precision: 0.6000, test  recall: 1.0000


Epoch: 0014, loss_train: 3.3833:   8%|▊         | 15/200 [00:03<00:19,  9.70it/s]


Mini Test for Epochs 10:
train loss: 0.574183, train accuracy: 0.7222, train f1 score: 0.8148, train confusion_matrix: [[ 2  5], [ 0 11]], train precision: 0.6875, train recall: 1.0000
test  loss: 0.578501, test  accuracy: 0.6000, test  f1 score: 0.7500, test  confusion_matrix: [[0 2], [0 3]], test  precision: 0.6000, test  recall: 1.0000


...

Epoch: 0184, loss_train: 1.2616:  92%|█████████▏| 184/200 [00:13<00:00, 19.31it/s]


Mini Test for Epochs 180:
train loss: 0.275615, train accuracy: 0.8889, train f1 score: 0.9167, train confusion_matrix: [[ 5  2], [ 0 11]], train precision: 0.8462, train recall: 1.0000
test  loss: 0.483484, test  accuracy: 0.8000, test  f1 score: 0.8571, test  confusion_matrix: [[1 1], [0 3]], test  precision: 0.7500, test  recall: 1.0000


Epoch: 0194, loss_train: 2.3016:  96%|█████████▋| 193/200 [00:13<00:00, 19.01it/s]


Mini Test for Epochs 190:
train loss: 0.591995, train accuracy: 0.8889, train f1 score: 0.9091, train confusion_matrix: [[ 6  1], [ 1 10]], train precision: 0.9091, train recall: 0.9091
test  loss: 0.646756, test  accuracy: 0.8000, test  f1 score: 0.8000, test  confusion_matrix: [[2 0], [1 2]], test  precision: 1.0000, test  recall: 0.6667


Epoch: 0199, loss_train: 2.1713: 100%|██████████| 200/200 [00:13<00:00, 14.30it/s]


In [ ]:
trainer.evaluate(cfg.epochs, train_loader, valid_loader)
vis_loader = DataLoader(all_graphs, shuffle=False, batch_size=1)
trainer.visualize_embeddings(vis_loader, "./")


Mini Test for Epochs 200:
train loss: 0.183219, train accuracy: 0.9444, train f1 score: 0.9565, train confusion_matrix: [[ 6  1], [ 0 11]], train precision: 0.9167, train recall: 1.0000
test  loss: 0.084670, test  accuracy: 1.0000, test  f1 score: 1.0000, test  confusion_matrix: [[2 0], [0 3]], test  precision: 1.0000, test  recall: 1.0000
best  loss: 0.080683, best  accuracy: 1.0000, best  f1 score: 1.0000, best  confusion_matrix: [[2 0], [0 3]], best  precision: 1.0000, best  recall: 1.0000


Use Case 3:

In [12]:
args = [
    '--yaml_path', '/content/drive/MyDrive/HW2VEC/examples/example_gnn4ip.yaml',
    '--raw_dataset_path', '/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/',
    '--data_pkl_path', '/content/drive/MyDrive/HW2VEC/examples/dgf_ip_rtl.pkl',
    '--graph_type', 'DFG',
    '--device', 'cuda'
    ]
cfg = Config(args)

In [13]:
''' prepare graph data '''
print(cfg.raw_dataset_path)
for verilog_path in glob("%s/**/*.v" % str(cfg.raw_dataset_path), recursive=True):
    folder_name = Path(verilog_path).parent
    print(folder_name)

''' converting graph using hw2graph '''
nx_graphs = []
hw2graph = HW2GRAPH(cfg)
for hw_project_path in hw2graph.find_hw_project_folders():
    hw_graph = hw2graph.code2graph(hw_project_path)
    print(hw_graph)
    nx_graphs.append(hw_graph)

data_proc = DataProcessor(cfg)
for hw_graph in nx_graphs:
    data_proc.process(hw_graph)
data_proc.cache_graph_data(cfg.data_pkl_path)

/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_3
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_1
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_2
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_4
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_5
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_6
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/adder4bit/adder4bit_7
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/bcdToseg/bcdToseg_4
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/bcdToseg/bcdToseg_3
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/bcdToseg/bcdToseg_6
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/bcdToseg/bcdToseg_2
/content/drive/MyDrive/HW2VEC/assets/datasets/IP-RTL-toy/bcdToseg/bcdToseg_8
/cont

In [14]:
SIMILAR = 1
DISSIMILAR = -1

data_proc.generate_pairs()
all_pairs = data_proc.get_pairs()
for pair_idx, pair in enumerate(all_pairs):
    graph_a, graph_b = pair
    if graph_a.hw_type == graph_b.hw_type:
        all_pairs[pair_idx] += (SIMILAR,)
    else:
        all_pairs[pair_idx] += (DISSIMILAR,)

train_pairs, test_pairs = data_proc.split_dataset(cfg.ratio, cfg.seed, all_pairs)
train_loader = DataLoader(train_pairs, shuffle=True, batch_size=cfg.batch_size)
test_loader  = DataLoader(test_pairs, shuffle=True, batch_size=cfg.batch_size)

In [15]:
''' model configuration '''
model = GRAPH2VEC(cfg)

convs = [
    GRAPH_CONV("gcn", data_proc.num_node_labels, cfg.hidden),
    GRAPH_CONV("gcn", cfg.hidden, cfg.hidden)
]
model.set_graph_conv(convs)

pool = GRAPH_POOL("sagpool", cfg.hidden, cfg.poolratio)
model.set_graph_pool(pool)

readout = GRAPH_READOUT("max")
model.set_graph_readout(readout)

output = nn.Linear(cfg.hidden, cfg.embed_dim)
model.set_output_layer(output)

In [ ]:
''' training '''
model.to(cfg.device)
trainer = PairwiseGraphTrainer(cfg)
trainer.build(model)
trainer.train(train_loader, test_loader)

Epoch: 0000, loss_train: 0.9986:   0%|          | 1/200 [00:00<01:26,  2.31it/s]


Mini Test for Epochs 0:
train loss: 0.350956, train accuracy: 0.3036, train f1 score: 0.4658, train confusion_matrix: [[  0 117], [  0  51]], train precision: 0.3036, train recall: 1.0000
test  loss: 0.345022, test  accuracy: 0.3095, test  f1 score: 0.4727, test  confusion_matrix: [[ 0 29], [ 0 13]], test  precision: 0.3095, test  recall: 1.0000


Epoch: 0011, loss_train: 0.5092:   6%|▌         | 12/200 [00:02<00:31,  5.94it/s]


Mini Test for Epochs 10:
train loss: 0.129461, train accuracy: 0.7619, train f1 score: 0.6970, train confusion_matrix: [[82 35], [ 5 46]], train precision: 0.5679, train recall: 0.9020
test  loss: 0.185813, test  accuracy: 0.6667, test  f1 score: 0.5882, test  confusion_matrix: [[18 11], [ 3 10]], test  precision: 0.4762, test  recall: 0.7692


Epoch: 0021, loss_train: 0.4417:  11%|█         | 22/200 [00:03<00:29,  6.11it/s]

...

Epoch: 0180, loss_train: 0.2329:  90%|█████████ | 181/200 [00:27<00:04,  3.94it/s]


Mini Test for Epochs 180:
train loss: 0.111336, train accuracy: 0.8095, train f1 score: 0.7419, train confusion_matrix: [[90 27], [ 5 46]], train precision: 0.6301, train recall: 0.9020
test  loss: 0.159456, test  accuracy: 0.6905, test  f1 score: 0.6061, test  confusion_matrix: [[19 10], [ 3 10]], test  precision: 0.5000, test  recall: 0.7692


Epoch: 0191, loss_train: 0.2238:  96%|█████████▌| 192/200 [00:29<00:01,  4.69it/s]


Mini Test for Epochs 190:
train loss: 0.108047, train accuracy: 0.8095, train f1 score: 0.7419, train confusion_matrix: [[90 27], [ 5 46]], train precision: 0.6301, train recall: 0.9020
test  loss: 0.163606, test  accuracy: 0.6905, test  f1 score: 0.6061, test  confusion_matrix: [[19 10], [ 3 10]], test  precision: 0.5000, test  recall: 0.7692


Epoch: 0199, loss_train: 0.2230: 100%|██████████| 200/200 [00:30<00:00,  6.49it/s]
